In [1]:
import numpy as np

Testing code using randomly generated matrices

In [13]:
import PCA_implementations as algorithms
import PCA_metrics as metrics

A = np.random.rand(20, 30)

In [14]:
k = 10
PCA_A = algorithms.k_dim_PCA(np.matmul(np.transpose(A), A), 10)


In [15]:
epsilon_range = [0.01, 0.1, 0.2, 0.5, 1, 2, 3, 4, 5]
delta_range = [0.001, 0.01, 0.1, 0.5, 1]

In [16]:
for epsilon in epsilon_range:
    print("epsilon: {}".format(epsilon))
    for delta in delta_range:
        print("\tdelta: {}".format(delta))
        alg_1_PCA = algorithms.microsoft_alg_1(A,epsilon, delta, k)
        print("\t\t{}".format(metrics.subspace_dist(PCA_A, alg_1_PCA)))

epsilon: 0.01
	delta: 0.001
		3618925.069376362
	delta: 0.01
		2557586.9021760095
	delta: 0.1
		1300211.4682345847
	delta: 0.5
		490792.91672421887
	delta: 1
		123894.74489401847
epsilon: 0.1
	delta: 0.001
		38188.39061234413
	delta: 0.01
		25534.197893903674
	delta: 0.1
		13343.893324459656
	delta: 0.5
		4698.894105646028
	delta: 1
		1231.6651065810786
epsilon: 0.2
	delta: 0.001
		9394.589977639547
	delta: 0.01
		6254.076686774682
	delta: 0.1
		3245.424045782867
	delta: 0.5
		1189.6207959550395
	delta: 1
		321.9725612457319
epsilon: 0.5
	delta: 0.001
		1454.535383515317
	delta: 0.01
		990.484483241639
	delta: 0.1
		503.19637282783106
	delta: 0.5
		192.7282720272266
	delta: 1
		46.360243053136664
epsilon: 1
	delta: 0.001
		396.4891203466988
	delta: 0.01
		260.457004392898
	delta: 0.1
		134.83773527806076
	delta: 0.5
		47.494796544690836
	delta: 1
		12.553841707172676
epsilon: 2
	delta: 0.001
		89.42158565980439
	delta: 0.01
		60.25056068315442
	delta: 0.1
		35.28181543221563
	delta: 0.

In [3]:
import cv2
import sklearn

In [4]:
im = cv2.imread("data/monarch.png")
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#im = cv2.resize(im, (512, 512))
print(np.amin(im), np.amax(im))

17 245


In [9]:
Xhat = algorithms.k_dim_PCA(im, 100)
Xhat = cv2.normalize(Xhat, Xhat, 0, 1, cv2.NORM_MINMAX)

In [19]:
Xhat_approx = algorithms.microsoft_alg_1(im, 3, 0.5, 100)
Xhat_approx = cv2.normalize(Xhat_approx, Xhat_approx, 0, 1, cv2.NORM_MINMAX)

In [21]:
print(im.shape, Xhat.shape, Xhat_approx.shape)
cv2.imshow("regular", im)
cv2.waitKey(0)
cv2.imshow("pca image", Xhat)
cv2.waitKey(0)
cv2.imshow("pca image", Xhat_approx)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 768) (512, 768) (768, 768)
